In [1]:
import pandas as pd

In [25]:
import dgutils.pandas as dgp

/Users/alicegao/anaconda2/envs/plot_py3/lib/python3.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from utils_plot import make_plot_bb, predict_bounidng_box, array_clean_up

In [3]:
df = pd.read_pickle('result/rf_data_all_targets_1/pred_ep_20.pkl.gz')

In [4]:
df.columns

Index(['subset', 'target_stem_on', 'pred_stem_on', 'target_stem_location_x',
       'pred_stem_location_x', 'target_stem_location_y',
       'pred_stem_location_y', 'target_stem_size', 'pred_stem_size',
       'target_iloop_on', 'pred_iloop_on', 'target_iloop_location_x',
       'pred_iloop_location_x', 'target_iloop_location_y',
       'pred_iloop_location_y', 'target_iloop_size_x', 'pred_iloop_size_x',
       'target_iloop_size_y', 'pred_iloop_size_y', 'target_hloop_on',
       'pred_hloop_on', 'target_hloop_location_x', 'pred_hloop_location_x',
       'target_hloop_location_y', 'pred_hloop_location_y', 'target_hloop_size',
       'pred_hloop_size'],
      dtype='object')

In [5]:
row = df.iloc[10]

In [6]:
target = row['target_stem_on']
pred_on = row['pred_stem_on']
pred_loc_x = row['pred_stem_location_x']
pred_loc_y = row['pred_stem_location_y']
pred_siz_x = row['pred_stem_size']

target, pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, m = array_clean_up(target, pred_on, pred_loc_x,
                                                                                     pred_loc_y, pred_siz_x, pred_siz_y=None)

bb_stem, pred_box = predict_bounidng_box(pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, thres=0.4)


In [7]:
target = row['target_iloop_on']
pred_on = row['pred_iloop_on']
pred_loc_x = row['pred_iloop_location_x']
pred_loc_y = row['pred_iloop_location_y']
pred_siz_x = row['pred_iloop_size_x']
pred_siz_y = row['pred_iloop_size_y']

target, pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, m = array_clean_up(target, pred_on, pred_loc_x,
                                                                                     pred_loc_y, pred_siz_x, pred_siz_y)

bb_iloop, pred_box = predict_bounidng_box(pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, thres=0.4)

In [8]:
target = row['target_hloop_on']
pred_on = row['pred_hloop_on']
pred_loc_x = row['pred_hloop_location_x']
pred_loc_y = row['pred_hloop_location_y']
pred_siz_x = row['pred_hloop_size']

target, pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, m = array_clean_up(target, pred_on, pred_loc_x,
                                                                                     pred_loc_y, pred_siz_x, pred_siz_y=None)

bb_hloop, pred_box = predict_bounidng_box(pred_on, pred_loc_x, pred_loc_y, pred_siz_x, pred_siz_y, thres=0.4)

In [24]:
def bb_corner(bb_x, bb_y, siz_x, siz_y):
    x0 = bb_x
    x1 = bb_x + siz_x - 1
    y0 = bb_y - siz_y + 1
    y1 = bb_y
    return x0, x1, y0, y1

In [74]:
df_stem = pd.DataFrame(bb_stem)
df_stem = df_stem[['bb_x', 'bb_y', 'siz_x', 'siz_y']].drop_duplicates()
df_stem = dgp.add_columns(df_stem, ['x0', 'x1', 'y0', 'y1'], ['bb_x', 'bb_y', 'siz_x', 'siz_y'], bb_corner)
df_stem['id'] = df_stem.index

df_iloop = pd.DataFrame(bb_iloop)
df_iloop = df_iloop[['bb_x', 'bb_y', 'siz_x', 'siz_y']].drop_duplicates()
df_iloop = dgp.add_columns(df_iloop, ['x0', 'x1', 'y0', 'y1'], ['bb_x', 'bb_y', 'siz_x', 'siz_y'], bb_corner)
df_iloop['id'] = df_iloop.index

df_hloop = pd.DataFrame(bb_hloop)
df_hloop = df_hloop[['bb_x', 'bb_y', 'siz_x', 'siz_y']].drop_duplicates()
df_hloop = dgp.add_columns(df_hloop, ['x0', 'x1', 'y0', 'y1'], ['bb_x', 'bb_y', 'siz_x', 'siz_y'], bb_corner)
df_hloop['id'] = df_hloop.index

In [76]:
def find_compatible_bb(df_1, df_2, inner=False, outer=False):
    assert inner or outer
    
    def _find_bb(d1, d2, direction):
        # join on compatible corner
        if direction == 'inner':
            # bottom left corner of 1 is identical to top right corner of 1
            df_t = pd.merge(df_1[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id', 'x1', 'y0']], 
                 df_2[['id', 'x0', 'y1']], 
                 left_on=['x1', 'y0'], right_on=['x0', 'y1'], how='inner')
        elif direction == 'outer':
            # top right corner of 1 is identical to bottom left corner of 2
            df_t = pd.merge(df_1[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id', 'x0', 'y1']], 
                 df_2[['id', 'x1', 'y0']], 
                 left_on=['x0', 'y1'], right_on=['x1', 'y0'], how='inner')
        else:
            raise ValueError
        df_t = df_t.rename(columns={'id_x': 'id', 'id_y': 'id_{}'.format(direction)})
        df_t = df_t[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id', 'id_{}'.format(direction)]]
        df_t = df_t.groupby(by=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id'])['id_{}'.format(direction)].apply(list).reset_index(name='id_{}'.format(direction))
    
        return df_t
    
    if inner:
        df_t = _find_bb(df_1, df_2, 'inner')
        if len(df_t) > 0:
            df_1 = pd.merge(df_1, df_t, on=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id'], how='left')
        else:
            # add a column of all None's
            df_1['id_inner'] = None

    if outer:
        df_t = _find_bb(df_1, df_2, 'outer')
        if len(df_t) > 0:
            df_1 = pd.merge(df_1, df_t, on=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id'], how='left')
        else:
            # add a column of all None's
            df_1['id_outer'] = None
    return df_1
    
    

In [87]:

# discard invalid hloop
# a hloop is valid if it's bounding box is symmetric across diagonal
df_hloop_2 = df_hloop[df_hloop['x0'] == df_hloop['y0']]

# for each hloop, find compatible stem(s) where:
# - stem is outer-touching hloop
# discard those with no compatible stems
find_compatible_bb(df_hloop_2, df_stem, outer=True).dropna(subset=['id_outer'])


,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id,id_outer
0,14,21,8,8,14,21,14,21,0,[1]
1,28,33,6,6,28,33,28,33,1,"[3, 7]"
2,44,49,6,6,44,49,44,49,2,[11]


In [88]:
# for each iloop, find compatible stems on each side: 
# - stem(s) outer-touching iloop
# - stem(s) inner-touching iloop
# discard those with no compatible stems (require both side to be non-empty)
find_compatible_bb(df_iloop, df_stem, inner=True, outer=True).dropna(subset=['id_inner']).dropna(subset=['id_outer'])

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id,id_inner,id_outer
9,40,54,2,3,40,41,52,54,9,[11],[10]


In [81]:

find_compatible_bb(df_stem, df_iloop, inner=True, outer=True)

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id,id_inner,id_outer
0,0,54,6,6,0,5,49,54,0,NaN,NaN
1,11,24,4,4,11,14,21,24,1,NaN,NaN
2,15,65,8,8,15,22,58,65,2,NaN,NaN
3,23,38,6,6,23,28,33,38,3,NaN,NaN
4,23,37,6,6,23,28,32,37,4,NaN,NaN
5,23,37,5,5,23,27,33,37,5,NaN,NaN
6,24,37,6,6,24,29,32,37,6,NaN,[1]
7,24,37,5,5,24,28,33,37,7,NaN,[1]
8,31,54,6,6,31,36,49,54,8,NaN,NaN
9,30,63,5,5,30,34,59,63,9,NaN,NaN


In [83]:
find_compatible_bb(df_stem, df_hloop, inner=True)

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id,id_inner
0,0,54,6,6,0,5,49,54,0,NaN
1,11,24,4,4,11,14,21,24,1,[0]
2,15,65,8,8,15,22,58,65,2,NaN
3,23,38,6,6,23,28,33,38,3,[1]
4,23,37,6,6,23,28,32,37,4,NaN
5,23,37,5,5,23,27,33,37,5,NaN
6,24,37,6,6,24,29,32,37,6,NaN
7,24,37,5,5,24,28,33,37,7,[1]
8,31,54,6,6,31,36,49,54,8,NaN
9,30,63,5,5,30,34,59,63,9,NaN


In [46]:
# for each hloop, find the compatible outer stem(s)
df_t = pd.merge(df_hloop[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id', 'x0', 'y1']], 
         df_stem[['id', 'x1', 'y0']], 
         left_on=['x0', 'y1'], right_on=['x1', 'y0'], how='left')
df_t = df_t.rename(columns={'id_x': 'id', 'id_y': 'id_stem'})
df_t = df_t[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id', 'id_stem']]
df_t = df_t.groupby(by=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'id'])['id_stem'].apply(list).reset_index(name='id_stem')

In [47]:
df_t

,bb_x,bb_y,siz_x,siz_y,id,id_stem
0,14,21,8,8,0,[1]
1,28,33,6,6,1,"[3, 7]"
2,44,49,6,6,2,[11]


In [31]:
df_stem

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id
0,0,54,6,6,0,5,49,54,0
1,11,24,4,4,11,14,21,24,1
2,15,65,8,8,15,22,58,65,2
3,23,38,6,6,23,28,33,38,3
4,23,37,6,6,23,28,32,37,4
5,23,37,5,5,23,27,33,37,5
6,24,37,6,6,24,29,32,37,6
7,24,37,5,5,24,28,33,37,7
8,31,54,6,6,31,36,49,54,8
9,30,63,5,5,30,34,59,63,9


In [32]:
df_iloop

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id
0,21,38,3,3,21,23,36,38,0
1,22,39,3,3,22,24,37,39,1
2,26,54,6,2,26,31,53,54,2
3,27,54,7,2,27,33,53,54,3
4,27,54,6,2,27,32,53,54,4
5,27,54,3,2,27,29,53,54,5
6,25,54,3,2,25,27,53,54,6
7,40,52,2,3,40,41,50,52,7
8,40,53,2,3,40,41,51,53,8
9,40,54,2,3,40,41,52,54,9


In [33]:
df_hloop

,bb_x,bb_y,siz_x,siz_y,x0,x1,y0,y1,id
0,14,21,8,8,14,21,14,21,0
1,28,33,6,6,28,33,28,33,1
2,44,49,6,6,44,49,44,49,2


In [12]:
# TODO 

# try other structures - done
# print predicted box location
# calculate box overlap, TP, FP, etc. (sensitivity, specificity)
# speed up